In [13]:
pip install pandas openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]
Note: you may need to restart the kernel to use updated packages.


In [36]:
# Inserting libraries to work
import pandas as pd
from pathlib import Path

In [27]:
# Load the dataframe
db_23 = pd.read_csv('/Volumes/Toshi 1Tb/Trade/Trade Correo/tradeInGoods_baci_2023.csv', sep=";", encoding='latin1')
db_23["year"] = "2023"

In [30]:
# Dropping unnecesary columns for speed. Dictionary is in excel
db_23_clean = db_23.drop(columns=['country_desciption', 'partner_desciption', 'item_description'])

# Changing european , for .
db_23_clean["value"] = (db_23_clean["value"].astype(str).str.replace(",", ".", regex=False))
db_23_clean["value"] = pd.to_numeric(db_23_clean["value"], errors="coerce")

# Creating new variable that is the first two digits of the item code
db_23_clean["item2"] = db_23_clean["item_code"].astype(str).str[:2]

In [31]:
#Splitting database into exporter/importer
db_23_imp = db_23_clean[db_23_clean["flow"] == "I"].copy()
db_23_exp = db_23_clean[db_23_clean["flow"] == "E"].copy()

In [ ]:
# Group at the 2 digit level
grouped_exp = (db_23_exp.groupby(["year", "country_code", "partner_code", "item2"], as_index=False)["value"].sum())
grouped_imp = (db_23_imp.groupby(["year", "country_code", "partner_code", "item2"], as_index=False)["value"].sum())

In [ ]:
# Base Code before loop to test and replicate in for-loop

# Load the dataframe
db_23 = pd.read_csv('/Volumes/Toshi 1Tb/Trade/Trade Correo/tradeInGoods_baci_2023.csv', sep=";", encoding='latin1')
db_23["year"] = "2023"

# Dropping unnecesary columns for speed. Dictionary is in excel
db_23_clean = db_23.drop(columns=['country_desciption', 'partner_desciption', 'item_description'])

# Changing european , for .
db_23_clean["value"] = (db_23_clean["value"].astype(str).str.replace(",", ".", regex=False))
db_23_clean["value"] = pd.to_numeric(db_23_clean["value"], errors="coerce")

# Creating new variable that is the first two digits of the item code
db_23_clean["item2"] = db_23_clean["item_code"].astype(str).str[:2]

#Splitting database into exporter/importer
db_23_imp = db_23_clean[db_23_clean["flow"] == "I"].copy()
db_23_exp = db_23_clean[db_23_clean["flow"] == "E"].copy()

# Group at the 2 digit level
grouped_exp = (db_23_exp.groupby(["year", "country_code", "partner_code", "item2"], as_index=False)["value"].sum())
grouped_imp = (db_23_imp.groupby(["year", "country_code", "partner_code", "item2"], as_index=False)["value"].sum())

,year,country_code,partner_code,item2,value
0,2023,ABW,AGO,61,5081.0
1,2023,ABW,AGO,84,1519.0
2,2023,ABW,ARE,10,408.0
3,2023,ABW,ARE,22,160.0
4,2023,ABW,ARE,30,6969.0


In [ ]:
# Folder containing all the CSV files
data_path = Path('/Volumes/Toshi 1Tb/Trade/Trade Correo/')

# Years you want to process
years = range(1996, 2023)

all_exports = []
all_imports = []

for year in years:
    print(f"Processing {year}...")

    # ========== 1. Load CSV ==========
    file_path = data_path / f"tradeInGoods_baci_{year}.csv"
    db = pd.read_csv(file_path, sep=";", encoding='latin1')

    # Add year column
    db["year"] = str(year)

    # ========== 2. Drop unnecessary columns ==========
    db_clean = db.drop(columns=[
        'country_desciption',
        'partner_desciption',
        'item_description'
    ])

    # ========== 3. Fix decimal commas ==========
    db_clean["value"] = (
        db_clean["value"]
        .astype(str)
        .str.replace(",", ".", regex=False)
    )
    db_clean["value"] = pd.to_numeric(db_clean["value"], errors="coerce")

    # ========== 4. Extract first two digits of item code ==========
    db_clean["item2"] = db_clean["item_code"].astype(str).str[:2]

    # ========== 5. Split into importer / exporter ==========
    db_imp = db_clean[db_clean["flow"] == "I"].copy()
    db_exp = db_clean[db_clean["flow"] == "E"].copy()

    # ========== 6. Group at 2-digit item level ==========
    grouped_exp = (
        db_exp.groupby(["year", "country_code", "partner_code", "item2"], as_index=False)["value"]
        .sum()
    )

    grouped_imp = (
        db_imp.groupby(["year", "country_code", "partner_code", "item2"], as_index=False)["value"]
        .sum()
    )

    # Store results in lists
    all_exports.append(grouped_exp)
    all_imports.append(grouped_imp)

# ========== 7. Concatenate results across all years ==========
exports_final = pd.concat(all_exports, ignore_index=True)
imports_final = pd.concat(all_imports, ignore_index=True)

# Optional: save results
exports_final.to_csv(data_path / "exports_1996_2023.csv", index=False)
imports_final.to_csv(data_path / "imports_1996_2023.csv", index=False)

print("Done!")

Processing 2023...
Done!
